In [1]:
import pandas as pd
import plotly.graph_objects as go

In [9]:
# Load data
data_noise = pd.read_csv("../Data for visualization/combined_noise_event.csv", header=0, sep=',')
data_noise.head(150)

,description,result_timestamp,noise_event_laeq_primary_detected_certainty,noise_event_laeq_primary_detected_class,certainty_percentage,month,day,hour,minute,second,milliseconds,lamax,laeq
0,MP 03: Naamsestraat 62 Taste,2022-01-01 00:01:31.362,NaN,NaN,NaN,1,1,0,1,31,362,70.4,68.8
1,MP 03: Naamsestraat 62 Taste,2022-01-01 00:02:19.368,NaN,NaN,NaN,1,1,0,2,19,368,NaN,NaN
2,MP 03: Naamsestraat 62 Taste,2022-01-01 00:02:23.368,NaN,NaN,NaN,1,1,0,2,23,368,NaN,NaN
3,MP 03: Naamsestraat 62 Taste,2022-01-01 00:02:33.370,NaN,NaN,NaN,1,1,0,2,33,370,93.4,87.4
4,MP 03: Naamsestraat 62 Taste,2022-01-01 00:02:57.373,NaN,NaN,NaN,1,1,0,2,57,373,94.3,87.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,MP 03: Naamsestraat 62 Taste,2022-01-14 07:13:36.877,43.0,Transport road - Passenger car,43.0%,1,14,7,13,36,877,70.0,68.9
146,MP 03: Naamsestraat 62 Taste,2022-01-14 07:54:16.894,22.0,Transport road - Passenger car,22.0%,1,14,7,54,16,894,71.0,69.5
147,MP 03: Naamsestraat 62 Taste,2022-01-14 09:09:20.322,72.0,Transport road - Passenger car,72.0%,1,14,9,9,20,322,73.8,71.0
148,MP 03: Naamsestraat 62 Taste,2022-01-14 09:24:10.298,98.0,Transport road - Passenger car,98.0%,1,14,9,24,10,298,73.4,71.9


In [7]:
# Check the number of missing values in 'noise_source' column
missing_values_count = data_noise['noise_event_laeq_primary_detected_class'].isnull().sum()
print("Number of missing values in noise source column:", missing_values_count)


Number of missing values in 'noise_source' column: 3822


In [10]:
# Replace empty values with 'Missing'
data_noise['noise_event_laeq_primary_detected_class'] = data_noise['noise_event_laeq_primary_detected_class'].fillna('Missing')


In [17]:
# Compute average noise level (laeq) for each noise source category
data_noise = data_noise.groupby('noise_event_laeq_primary_detected_class')['laeq'].mean().reset_index()
# Sort the dataset based on 'laeq'
data_noise = data_noise.sort_values(by='laeq')
data_noise.head(10)

,noise_event_laeq_primary_detected_class,laeq
5,Transport road - Passenger car,69.070671
3,Music non-amplified,70.798889
2,Missing,71.035057
7,Unsupported,71.267030
0,Human voice - Shouting,71.804588
6,Transport road - Siren,71.823963
4,Nature elements - Wind,71.971605
1,Human voice - Singing,74.154955


In [20]:
# Create line trace
line_trace = go.Scatter(
    x=[0],
    y=[],
    mode='lines',
    line=dict(color='red', width=1),
    showlegend=False
)

# Create circle marker traces
circle_marker_traces = []
for i in range(len(data_noise)):
    circle_trace = go.Scatter(
        x=[data_noise['laeq'].iloc[i]],
        y=[data_noise['noise_event_laeq_primary_detected_class'].iloc[i]],
        mode='markers',
        marker=dict(
            size=data_noise['laeq'].iloc[i],
            sizemode='diameter',
            sizeref=0.2,
            sizemin=3,
        ),
        showlegend=False
    )
    circle_marker_traces.append(circle_trace)

# Combine line trace and circle marker traces
data = [line_trace] + circle_marker_traces

# Create layout
layout = go.Layout(
    title='Average Noise Level by Category',
    xaxis=dict(title='Average Noise Level (laeq)'),
    yaxis=dict(title='Noise Source', autorange='reversed'),
    height=600,
    width=800,
    margin=dict(l=100, r=50, t=100, b=50),
)

# Create figure
fig = go.Figure(data=data, layout=layout)

# Display the graph
fig.show()